In [1]:
import pandas as pd
import numpy as np
import supy as sp
from pathlib import Path
from datetime import datetime
import pickle

In [2]:
def spin_up(cityname,first_day_str):
    path_runcontrol = Path('../input-processor/pre-processor-UK/runs/run_'+cityname) / 'RunControl.nml'
    df_state_init = sp.init_supy(path_runcontrol)
    grid = df_state_init.index[0]
    df_forcing = sp.load_forcing_grid(path_runcontrol, grid)
    first_day = datetime.strptime(first_day_str, '%Y-%m-%d')

    print('Rotating the time based on the first day of '+first_day_str)
    first_part = df_forcing.loc[df_forcing[df_forcing.index >= first_day].index]
    second_part = df_forcing.loc[df_forcing[
        df_forcing.index < first_day-pd.Timedelta('1 days')
    ].index]
    second_part.index = second_part.index+pd.Timedelta('366 days')

    df_forcing_2 = first_part.append(second_part)
    df_forcing_2.iy = df_forcing_2.index.year
    df_forcing_2.index.freq = first_part.index.freq
    round_number = 0
    error = 0.4
    counter=0
    while (error >= 0.1 or counter<3):
        counter+=1
        round_number = round_number+1
        print(f'\r{round_number}-->{error}',end="")
        df_output, df_state_final = sp.run_supy(
            df_forcing_2, df_state_init,
            save_state=False)
        final_state = df_state_final[df_state_init.columns.levels[0]
                                     ].iloc[1]
        df_state_init.iloc[0] = final_state
        soilstore_before = df_state_final.soilstore_id.iloc[0]
        soilstore_after = df_state_final.soilstore_id.iloc[1]
        diff_soil = sum(abs(soilstore_after-soilstore_before))
        error = 100*diff_soil/soilstore_before.mean()

    
    return df_state_init, df_forcing_2


def run_save_suews(cityname,first_day_str,name_season):
    df_state_init, df_forcing=spin_up(cityname,first_day_str)
    
    if cityname=='London':
    #Modification based on WRF_SUEWS Preprocessing London part
        df_state_init.bldgh=21.11
        df_state_init.evetreeh=6.55
        df_state_init.dectreeh=6.55
        df_state_init.loc[:,'sfr']=[0.33088,0.3646,0.0,0.04956,0.0387959,0,0.21607] #sfr for grid contaning KCL
    
    df_output, df_state_final = sp.run_supy(df_forcing, df_state_init,save_state=False)
    out=df_output.loc[98].SUEWS
    
    with open(f'outputs/{cityname}-{name_season}-output-offline.pickle', 'wb') as handle:
        pickle.dump(out, handle, protocol=pickle.HIGHEST_PROTOCOL)
    

In [ ]:
citynames=['London','Swindon']
first_day_strs = ['2012-04-10','2012-07-15','2012-10-01','2012-01-10']
name_seasons = ['Apr','Jul','Oct','Jan']

for cityname in citynames:
    print(f'Running for {cityname} . . .')
    print('')
    print('.'*60)
    for name_season,first_day_str in zip(name_seasons,first_day_strs):
        run_save_suews(cityname,first_day_str,name_season)
        print('')
        print('.'*60)    
    print('#'*60)

Running for Swindon . . .

............................................................
Rotating the time based on the first day of 2012-04-10
4-->0.5847081799818272
............................................................
Rotating the time based on the first day of 2012-07-15
4-->0.24033435717299972
............................................................
Rotating the time based on the first day of 2012-10-01
4-->0.9371815814704477
............................................................
Rotating the time based on the first day of 2012-01-10
2-->20.22177053124844